<a href="https://colab.research.google.com/github/alonsogr03/Big-Data-Processing-I/blob/main/Practica_3_a_gonzalezr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚽ Práctica 3 - InfluxDB: Análisis de Datos de Rendimiento de Jugadores de Fútbol

### Objetivos de la Práctica
En esta práctica aprenderás a:
- Conectar y consultar datos en InfluxDB 3.
- Explorar y analizar la estructura del dataset de tracking de jugadores.
- Realizar agregaciones y consultas complejas usando SQL sobre series temporales.
- Aplicar análisis de ventanas fijas y móviles para estudiar la evolución temporal de la velocidad y frecuencia cardíaca.
- Interpretar los resultados para identificar patrones de rendimiento y carga física de los jugadores.

### Dataset

Trabajarás con un dataset de tracking de partidos de fútbol que incluye mediciones cada pocos segundos de 15 jugadores por partido. El dataset contiene:

- Datos del partido: `partido_id`, marca temporal `time`.
- Datos de los jugadores: `jugador_id`, posición en el campo (`pos_x`, `pos_y`).
- Datos de rendimiento físico: velocidad instantánea (`velocidad_kmh`), aceleración (`aceleracion_m_s2`), frecuencia cardíaca (`frecuencia_cardiaca_bpm`).

El dataset está disponible en la serie temporal `sensor_fatiga` de InfluxDB. Se incluyen múltiples partidos y todos los jugadores participantes.

A la hora de cargar los datos en InfluxDB 3, ten en cuenta que las columnas de tipo tag deben ser `partido_id` y `jugador_id`.

El dataset está disponible en el archivo `sensores_deportivos_fatiga.csv`. Se incluye, además, una descripción más detallada del dataset en el fichero `descripcion_sensores_deportivos_fatiga.txt`.

### Requisitos de implementación
- Se debe utilizar una cuenta de InfluxDB Cloud para la realización de la práctica.
- Todas las consultas se realizarán sobre InfluxDB usando SQL compatible con InfluxDB 3.
- Se evaluarán agregaciones, filtrados, agrupaciones, y uso de ventanas fijas (`DATE_BIN`) y móviles (`OVER ... RANGE INTERVAL`).
- Se permitirá convertir los resultados a Pandas únicamente para visualización de gráficos si fuese necesario.

### Entrega
Se entregarán los siguientes ficheros en el formulario de entrega habilitado:
- Este notebook, cambiándole el nombre a `Practica_3_usuario-urjc.ipynb` o `Practica_3_usuario-urjc1_usuario-urjc2.ipynb` en caso de entrega en pareja. Si tu correo es p.perez@urjc.es, el fichero se llamará `Practica_3_p.perez.ipynb` o `Practica_3_p.perez_j.lopez.ipynb`.
  
La fecha de entrega se comunicará en clase el día de la presentación de la práctica, así como en el foro de novedades y se podrá consultar también en el formulario de entrega.

---
## 📦 Configuración Inicial

In [ ]:
# Instalar dependencias (si es necesario)
!pip install influxdb3-python pandas matplotlib

In [ ]:
# Montamos la carpeta (nos pedirá permisos)
from google.colab import drive
drive.mount('/content/drive')
# Crea un atajo llamado 'workspace' en la carpeta /content (dará un pequeño error si ya existe)
!ln -s "/content/drive/MyDrive/Colab Notebooks" "/content/workspace" >/dev/null 2>&1
# Ya podemos acceder a los ficheros:
data_path = "/content/workspace/data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Configuración de la conexión a InfluxDB

# URL de la instancia de InfluxDB
HOST = "https://us-east-1-1.aws.cloud2.influxdata.com"
# Organización (solo necesario en InfluxDB Cloud)
ORG = "master"
# Base de datos (en InfluxDB Cloud se denomina bucket y es necesario crearlo antes)
DB = "practica3"
# Contraseña de acceso
TOKEN = "nkS1HSzHrbH9StTsC3vHE8ioHq10cmNPdJYy90HBEkZ3f8HBHBbnnKWTO72sZ-DfvivpWrcLvsMdY5WPZpCK-Q=="

In [ ]:
# Crear la conexión a InfluxDB
from influxdb_client_3 import InfluxDBClient3

client = InfluxDBClient3(host=HOST, token=TOKEN, org=ORG, database=DB)

In [ ]:
# Permitir la visualización completa de DataFrames en Pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 📊 Carga y Exploración de la serie temporal

Deberás cargar la serie temporal `sensores_deportivos_fatiga.csv` en un Bucket de InfluxDB Cloud.

In [ ]:
# Comenzamos cargando la serie temporal:
client.write_file(file=data_path + "sensores_deportivos_fatiga.csv", tag_columns=["partido_id", "jugador_id"],data_format="csv")


### 🔍 Exploración Básica

Antes de resolver los ejercicios, realiza una exploración básica del dataset: número de jugadores, número de partidos, distribución de registros por jugador, etc.

In [ ]:
# Vamos a comenzar mostrando el número de jugadores y de partidos:
query = """
SELECT COUNT(DISTINCT(jugador_id)) AS num_jugadores, COUNT(DISTINCT(partido_id)) AS num_partidos
FROM sensor_fatiga
 """
table = client.query(query=query)
df = table.to_pandas()
n_jugadores = df.iloc[0]['num_jugadores']
n_partidos = df.iloc[0]['num_partidos']

print(f"El dataset contiene información de {n_jugadores} jugadores y {n_partidos} partidos.")

# Vamos a visualizar ahora la distribución de registros por jugador:

query = """
SELECT jugador_id, COUNT(*) AS num_registros
FROM sensor_fatiga
GROUP BY jugador_id
ORDER BY num_registros DESC
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)


El dataset contiene información de 15 jugadores y 3 partidos.


,jugador_id,num_registros
0,8,720
1,6,720
2,11,720
3,2,720
4,3,720
5,9,720
6,15,720
7,5,720
8,4,720
9,1,720


Observamos que se han recogido datos sobre 15 jugadores en un total de 3 partidos, obteniendo en total 720 registros para cada jugador.

## 🎯 EJERCICIO 1: Análisis de Rendimiento Físico por Jugador y Partido

**Calificación:** 2.5 puntos

**Objetivo**: Calcular métricas de rendimiento físico para cada jugador en cada partido.

Para cada combinación de **`partido_id`** y **`jugador_id`**, calcula:

1. **Número total de registros** (`total_registros`)  
2. **Velocidad máxima alcanzada en km/h** (`velocidad_maxima`)  
3. **Velocidad media en km/h** (`velocidad_media`)  
4. **Frecuencia cardíaca media en bpm** (`fc_media`)  
5. **Filtra** solo los jugadores cuya **frecuencia cardíaca media sea superior a 140 bpm** durante el partido.

**Muestra** los resultados ordenados por:

- `fc_media` (descendente)  
- `velocidad_media` (descendente)

**💡 Pistas**:

- Usa `GROUP BY partido_id, jugador_id`  
- Las funciones de agregación útiles son: `COUNT()`, `MAX()`, `AVG()`  
- Usa `HAVING` para filtrar según la frecuencia cardíaca media  

### 📝 Solución

In [ ]:
# Solución ejercicio 1 aquí
query = """
SELECT partido_id, jugador_id, COUNT(*) as total_registros, MAX(velocidad_kmh) as velocidad_maxima, AVG(velocidad_kmh) as velocidad_media, AVG(frecuencia_cardiaca_bpm) as fc_media
FROM sensor_fatiga
GROUP BY partido_id, jugador_id
HAVING AVG(frecuencia_cardiaca_bpm) > 140
ORDER BY fc_media DESC, velocidad_media DESC
 """
table = client.query(query=query)
df = table.to_pandas()
display(df)


,partido_id,jugador_id,total_registros,velocidad_maxima,velocidad_media,fc_media
0,1,11,240,29.957136,16.773121,143.970833
1,3,10,240,29.762085,16.639787,142.000000
2,1,3,240,29.997781,17.620126,141.962500
3,1,10,240,29.852080,17.918909,141.354167
4,2,6,240,29.942277,17.425019,141.241667
5,3,13,240,29.821984,17.459966,141.175000
6,2,12,240,29.858514,18.023318,141.166667
7,2,2,240,29.888148,17.381114,141.150000
8,1,4,240,29.947756,18.374868,141.145833
9,1,15,240,29.906250,18.294304,140.958333


#### Pregunta de reflexión 1. ¿Qué jugador presenta mayor estrés fisiológico durante los partidos?

Consulta los resultados del **Ejercicio 1**. El jugador con mayor `fc_media` (frecuencia cardíaca media) refleja un mayor estrés fisiológico durante el partido. Compara esta información con `velocidad_maxima` y `velocidad_media` para analizar si los jugadores más rápidos tienden a tener mayores cargas cardíacas.

### Resolución
En la tabla anterior, se puede observar como el jugador con *jugador_id=11* fue el individuo que presentó mayor estrés fisiológico durante el partido *partido_id=1*.

Comparando la información de la frecuencia cardíaca media respecto de la velocidad máxima y velocidad media, no existe evidencia que haga indicar que los jugadores más rápidos tienen mayores cargas cardíacas, ya que como se puede observar en la tabla anterior, jugadores como el *4 y 15* tienen hasta 3 pulsaciones menos en media durante el partido y aún así, mayor velocidad media que el jugador *11*. De manera análoga con la velocidad máxima, hay jugadores con 2/3 pulsaciones menos en media con una mayor velocidad máxima.

Por lo tanto se puede concluir que tomando resultados sobre distintos jugadores, no existe relación positiva entre la frecuencia cardiaca y la velocidad. Al contrario, si sólamente evaluáramos a un sólo sujeto, entonces sí que existiría esta relación positiva.

---
## 🎯 EJERCICIO 2: Análisis de Actividad y Movimiento de Jugadores

**Calificación:** 2.5 puntos

**Objetivo**: Analizar la **actividad y rango de movimiento** de los jugadores durante los partidos.

Para cada combinación de **`partido_id`** y **`jugador_id`**, calcula:

1. **Número total de registros** (`total_registros`)  
2. **Velocidad máxima alcanzada en km/h** (`velocidad_maxima`)  
3. **Velocidad media en km/h** (`velocidad_media`)  
4. **Rango de posiciones en X** (`rango_pos_x`) → `MAX(pos_x) - MIN(pos_x)`  
5. **Rango de posiciones en Y** (`rango_pos_y`) → `MAX(pos_y) - MIN(pos_y)`

**Filtra** solo a los jugadores cuyo **rango de posiciones en X sea mayor a 40 metros** (jugadores más activos a lo largo del campo).

**Muestra** los resultados ordenados por:

- `rango_pos_x` (descendente)  
- `velocidad_maxima` (descendente)

**💡 Pistas**:

- Usa `GROUP BY partido_id, jugador_id`  
- Las funciones de agregación útiles son: `COUNT()`, `MAX()`, `MIN()`, `AVG()`  
- Usa `HAVING` para filtrar según el rango de posiciones en X

### 📝 Solución

In [ ]:
# Solución ejercicio 2 aquí
query = """
SELECT partido_id, jugador_id, COUNT(*) AS total_registros, MAX(velocidad_kmh) AS velocidad_maxima, AVG(velocidad_kmh) AS velocidad_media, MAX(pos_x) - MIN(pos_x) AS rango_pos_x,
MAX(pos_y) - MIN(pos_y) AS rango_pos_y, (MAX(pos_x)- MIN(pos_x))+(MAX(pos_y)- MIN(pos_y)) AS rango_total
FROM sensor_fatiga
GROUP BY partido_id, jugador_id
HAVING MAX(pos_x) - MIN(pos_x) > 40
ORDER BY rango_pos_x DESC, velocidad_maxima DESC
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)

,partido_id,jugador_id,total_registros,velocidad_maxima,velocidad_media,rango_pos_x,rango_pos_y,rango_total
0,1,13,240,29.886234,18.026688,99.953487,59.286318,159.239804
1,3,2,240,29.956238,18.812509,99.901144,59.508350,159.409493
2,2,12,240,29.858514,18.023318,99.828486,59.428035,159.256521
3,2,1,240,29.766566,18.036094,99.818329,59.866767,159.685097
4,2,7,240,28.671262,14.007701,99.744545,59.637318,159.381863
5,2,4,240,29.941276,17.668435,99.693771,59.919852,159.613623
6,1,12,240,29.928606,17.413155,99.647918,58.912688,158.560607
7,3,3,240,29.935586,17.404158,99.570978,59.666089,159.237066
8,1,2,240,29.986948,16.963286,99.569999,59.579022,159.149021
9,1,1,240,29.990831,17.494100,99.563738,59.336572,158.900310


#### Pregunta de reflexión 2. ¿Qué jugadores son los más activos en el campo y cómo se relaciona con su velocidad?

Consulta los resultados del **Ejercicio 2**. Identifica los jugadores con mayor `rango_pos_x` y `rango_pos_y`, es decir, los que recorrieron más distancia horizontal y vertical durante el partido.  

Reflexiona sobre lo siguiente:  
- ¿Los jugadores más activos también presentan mayor `velocidad_maxima` o `velocidad_media`?  
- ¿Hay jugadores con un rango de movimiento amplio pero velocidades relativamente bajas?  
- ¿Qué implicaciones podría tener esto sobre su carga física y posición táctica en el campo?

#### Resolución

Los 3 jugadores que recorrieron mayor distancia horizontal y vertical durante el partido fueron:

- Jugador 1 en el partido 2.
- Jugador 4 en el partido 2.
- Jugador 1 en el partido 3.

Los jugadores más activos no presentan una mayor velocidad_maxima o velocidad_media, por ejemplo, jugadores como el *7 en el partido 2*, situado en el top 5 de mayor distancia recorrida, es un jugador con una velocidad media 4 puntos por debajo que el jugador inmediatamente superior a él. Además, comparando a los 3 jugadores que más distancia recorrieron, existen jugadores mucho más abajo en el ránking con velocidades parecidas, lo cual parece indicar que no existe dicha relación.

En cuanto a jugadores con un rango de movimiento amplio pero velocidades bajas, como bien se ha mencionado antes, el jugador *7 en el partido 2* presenta una velocidad media de 14km/h aproximadamente, cuando la mayoría de valores de la tabla se encuentra entre los 17/18 km/h.

Estos resultados podrían implicar que aquellos jugadores que han recorrido una distancia similar a una menor velocidad, han absorbido una menor carga física, ya que si observamos su velocidad máxima, es muy parecida a la de los demás jugadores. Respecto a su posición táctica, es probable que este tipo de jugadores hayan ocupado posiciones de mediocampo como pivotes o organizadores del juego donde, gran parte del tiempo tienen que trotar y bascular por el terreno de juego pero pocas veces tienen que acumular metros a máxima/submáxima velocidad.

---
## 🎯 EJERCICIO 3: Conteo de Registros y Velocidad Promedio en Ventanas de 5 Minutos

**Calificación:** 2.5 puntos

**Objetivo**: Analizar la actividad de los jugadores en intervalos de tiempo fijos durante el partido.

Para cada jugador y partido, calcula en **ventanas fijas de 5 minutos**:

1. **Número de registros en la ventana** (`total_registros_5min`)  
2. **Velocidad promedio en la ventana** (`velocidad_media_5min`)  

**Muestra** los resultados ordenados por:

- `jugador_id`  
- `window_start` (inicio de la ventana)  

**💡 Pistas**:

- Usa `DATE_BIN(INTERVAL '5 minutes', timestamp, TIMESTAMP '1970-01-01T00:00:00Z')` para definir la ventana.  
- Agrupa por `partido_id`, `jugador_id` y la ventana (`window_start`).  
- Las funciones de agregación útiles son: `COUNT()` y `AVG()`.  
- Este ejercicio permite ver cómo varía la velocidad y la actividad del jugador en intervalos de tiempo fijos.


### 📝 Solución

In [ ]:
# Solución ejercicio 3 aquí
query = """
SELECT DATE_BIN(INTERVAL '5 minutes', time, TIMESTAMP '1970-01-01T00:00:00Z') as window_start,
partido_id, jugador_id, COUNT(*) AS total_registros_5min, AVG(velocidad_kmh) as velocidad_media_5min

from sensor_fatiga
GROUP BY partido_id, jugador_id, window_start
ORDER BY jugador_id, window_start
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)

,window_start,partido_id,jugador_id,total_registros_5min,velocidad_media_5min
0,2025-11-25 16:00:00,1,1,60,17.180867
1,2025-11-25 16:05:00,1,1,60,16.343655
2,2025-11-25 16:10:00,1,1,60,17.743044
3,2025-11-25 16:15:00,1,1,60,18.708835
4,2025-11-26 16:00:00,2,1,60,18.033468
5,2025-11-26 16:05:00,2,1,60,19.229328
6,2025-11-26 16:10:00,2,1,60,17.605463
7,2025-11-26 16:15:00,2,1,60,17.276120
8,2025-11-27 16:00:00,3,1,60,17.604840
9,2025-11-27 16:05:00,3,1,60,17.142035


#### Pregunta de reflexión 3. ¿Cómo varía la velocidad de los jugadores durante el partido?

Consulta los resultados del **Ejercicio 3**. Observa la columna `velocidad_media_5min` para cada ventana de 5 minutos.  

Reflexiona sobre lo siguiente:  
- ¿Hay ventanas donde los jugadores alcanzan picos de velocidad o muestran menor actividad?  
- ¿Qué jugador mantiene un ritmo más constante a lo largo del partido?  
- ¿Cómo podrían estas observaciones ayudar a planificar descansos, sustituciones o entrenamientos específicos?

#### Resolución

Observando la tabla creada, podemos ver que hay jugadores como el *jugador 1 en el partido 1*, que va aumentando la velocidad media a medida que se desarrolla el partido. Por otro lado, en el *segundo partido del jugador 1*, observamos que en la segunda ventana alcanza un pico de velocidad y a partir de ahí, su velocidad va decayendo poco a poco, probablemente causa de la fatiga de haber realizado una ventana a una mayor velocidad.


Para contestar a la pregunta de qué jugador mantiene un ritmo más constante a lo largo del partido, vamos a realizar la siguiente consulta con PANDAS. Se ha hecho la suposición de que buscamos al jugador más constante dentro de un sólo partido, no en todos los partidos jugados.

In [ ]:
constancia = df.groupby(['partido_id', 'jugador_id'])['velocidad_media_5min'].std().reset_index()
constancia.rename(columns={'velocidad_media_5min': 'desviacion_std'}, inplace=True)
ranking = constancia.sort_values(by='desviacion_std', ascending=True)
print("TOP JUGADORES MÁS CONSTANTES")
display(ranking.head())

TOP JUGADORES MÁS CONSTANTES


,partido_id,jugador_id,desviacion_std
36,3,15,0.263327
40,3,5,0.315876
22,2,2,0.355447
43,3,8,0.403643
29,2,9,0.444720


Por lo tanto, podemos observar que el jugador más constante se trata del *jugador_id=15* en el *partido_id=3*.

Este tipo de consultas de agregación temporal a través de ventanas fijas en series temporales, permite transformar datos en bruto en información táctica muy útil para entrenadores y preparadores físicos. Al segmentar el partido en ventanas fijas, se puede detectar fatigas agudas en los jugadores (jugadores que alcanzaron un pico de velocidad en alguna ventana y luego fue decayendo con el paso de las siguientes ventanas) durante el partido, siendo útiles para identificar posibles sustituciones por cansancio en directo durante el partido. Además, este tipo de consultas son también muy útiles para evaluar a los jugadores post-partido y así poder decidir cómo serán sus entrenamientos específicos de recuperación dependiendo cuál haya sido su rendimiento durante las distintas ventanas del partido.

---
## 🎯 EJERCICIO 4: Análisis de Velocidad Promedio con Ventana Móvil

**Calificación:** 2.5 puntos

**Objetivo**: Analizar la evolución de la velocidad de los jugadores en ventanas móviles de 10 minutos durante el partido.

Para cada jugador, calcula en **ventanas móviles de 10 minutos**:

1. **Velocidad promedio en la ventana** (`velocidad_media_10min`)  

**Muestra** los resultados ordenados por:

- `jugador_id`  
- `time` (marca temporal de cada registro)  

**💡 Pistas**:

- Usa `AVG(velocidad_kmh) OVER (...)` para calcular la media dentro de la ventana móvil.  
- `PARTITION BY jugador_id` permite calcular la ventana de forma independiente para cada jugador.  
- `ORDER BY time RANGE INTERVAL '10 minutes' PRECEDING` define la ventana móvil de 10 minutos.  
- Filtra los datos si quieres limitar el análisis al último día o al periodo de interés.  
- Este ejercicio permite ver cómo **varía la velocidad promedio en tiempo real** durante el partido.

### 📝 Solución

In [ ]:
# Solución ejercicio 4 aquí
query = """
SELECT time, jugador_id, velocidad_kmh as velocidad_instantanea,  AVG(velocidad_kmh) OVER (
  PARTITION BY jugador_id
  ORDER BY time RANGE INTERVAL '10 minutes' PRECEDING
) as velocidad_media_10min

from sensor_fatiga
WHERE partido_id = 1
ORDER BY jugador_id, time
"""
table = client.query(query=query)
df = table.to_pandas()
display(df)




,time,jugador_id,velocidad_instantanea,velocidad_media_10min
0,2025-11-25 16:00:00,1,21.944859,21.944859
1,2025-11-25 16:00:05,1,11.401044,16.672951
2,2025-11-25 16:00:10,1,10.327449,14.557784
3,2025-11-25 16:00:15,1,8.472258,13.036402
4,2025-11-25 16:00:20,1,17.328509,13.894824
5,2025-11-25 16:00:25,1,21.256151,15.121711
6,2025-11-25 16:00:30,1,18.956932,15.669600
7,2025-11-25 16:00:35,1,23.379970,16.633396
8,2025-11-25 16:00:40,1,11.287898,16.039452
9,2025-11-25 16:00:45,1,26.128201,17.048327


#### Pregunta de reflexión 4. ¿Cómo varía la velocidad promedio de los jugadores en ventanas móviles de 10 minutos?

Consulta los resultados del **Ejercicio 4**. Observa la columna `velocidad_media_10min` para cada jugador a lo largo del tiempo.

Reflexiona sobre lo siguiente:  
- ¿Qué jugadores muestran picos de velocidad más pronunciados en sus ventanas móviles?  
- ¿Hay jugadores que mantienen un ritmo constante durante todo el partido?  
- ¿Cómo podrían estos patrones ayudar a planificar sustituciones o descansos estratégicos durante el partido?

#### Resolución

Con el objetivo de visualizar mejor la evolución de cada jugador durante el primer partido jugado (partido_id=1), vamos a graficar sus ventanas móviles:


In [ ]:
import plotly.express as px
import pandas as pd


df['time'] = pd.to_datetime(df['time'])
df['jugador_id'] = df['jugador_id'].astype(str)
fig = px.line(
    df,
    x='time',
    y='velocidad_media_10min',
    color='jugador_id',
    title='Evolución de la Velocidad Media (Ventana Deslizante 10 min)',
    labels={
        'time': 'Hora del Partido',
        'velocidad_media_10min': 'Velocidad Media (km/h)',
        'jugador_id': 'Jugador'
    },
    color_discrete_sequence=px.colors.qualitative.Dark24
)

fig.update_xaxes(
    tickformat="%H:%M",
    title_text='Hora del Partido'
)

fig.update_layout(
    hovermode="x unified",
    legend=dict(
        title="Jugadores (Click para filtrar)",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01
    )
)

fig.show()

En la gráfica podemos obervar como el jugador 3, durante los 5 primeros minutos del partido, es el jugador con mayor pico de velocidad en la ventana móvil.

Se observa cómo la mayoría de jugadores mantienen una velocidad bastante constante durante todo el partido, destacando el jugador 7, cuya velocidad media en la ventana, va disminuyendo a partir del minuto 10 de forma bastante aguda, alcanzando una velocidad media en la ventana móvil del último minuto de unos 12km/h.

Este tipo de ventanas móviles es muy útil en eventos en directo ya que, si evaluamos simplemente ventanas fijas o el dato instantáneo, podrías ver picos basados en parones del juego, o tramos de partido menos exigentes para el jugador. Las ventanas móviles suavizan esto y permiten detectar patrones fisiológicos de fatiga (si miramos la velocidad como es este caso) que de otra forma serían invisibles. De esta forma, podemos determinar de mejor manera cuándo un jugador comienza a estar fatigado y de esta forma, darle descanso a través de una sustitución.





---
## 🧹 Limpieza Final

In [ ]:
client.close()